
# 02a — Model comparison & key insights  

Сравниваем **6 ML‑моделей** по PR‑AUC, смотрим feature‑importance LGBM  
и демонстрируем парадокс «богатый клиент уходит чаще».


In [1]:

from pathlib import Path
import pandas as pd
import joblib, json, matplotlib.pyplot as plt, seaborn as sns
sns.set_style("whitegrid")

# ► каталоги
ROOT = Path.cwd().parent          # репозиторий (notebooks/ -> ../)
ART  = ROOT / "artifacts"         # Optuna и прочие артефакты

# ► метрики PR‑AUC 6 моделей
metrics = {
    "LogReg": 0.474,
    "RandomForest": 0.505,
    "GradientBoosting": 0.521,
    "MLP": 0.466,
    "LightGBM (tuned)": 0.586,
    "CatBoost": 0.562,
}
pr_auc = pd.Series(metrics).sort_values()

# ► Optuna‑study + лучшие параметры
study       = joblib.load(ART / "lgbm_optuna.pkl")
best_params = json.load(open(ART / "lgbm_best_params.json"))
print('Optuna best PR‑AUC', round(study.best_value, 5))


c:\Users\imoney\projects\HSE2206\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\imoney\\projects\\HSE2206\\artifacts\\lgbm_best_params.json'

In [ ]:

ax = pr_auc.plot.barh(figsize=(6,4), color="skyblue")
ax.set_xlabel("PR‑AUC"); ax.set_title("Сравнение 6 моделей")
for i,v in enumerate(pr_auc.values):
    ax.text(v + 0.002, i, f"{v:.3f}", va="center")
plt.show()


In [ ]:

from lightgbm import LGBMClassifier, plot_importance

# обучаем заново (оба препроцессора внутри train_churn уже есть)
X, y = pd.read_csv(ROOT/'data/raw/churn/Churn_Modelling.csv', index_col=False), None
y = X.pop('Exited').values

best_lgbm = LGBMClassifier(**best_params).fit(X, y)
plot_importance(best_lgbm, max_num_features=15, figsize=(6,4))
plt.title("LightGBM Feature Importance (top‑15)")
plt.show()


In [ ]:

sns.boxplot(x='Exited', y='Balance',
            data=pd.read_csv(ROOT/'data/raw/churn/Churn_Modelling.csv'),
            palette='pastel')
plt.title("Парадокс: клиенты с высокими Balance уходят чаще")
plt.xlabel("Exited (0 = остался, 1 = ушёл)")
plt.ylabel("Balance (€)")
plt.show()



## Ключевые выводы  
* **LightGBM + Optuna** даёт прирост PR‑AUC ≈ **+0.11** к логрегрессии.  
* Тройка важнейших признаков: `Age`, `Balance`, `CreditScore`.  
* Boxplot показывает, что медианный **Balance ушедших выше** (≈ €145 k) — парадокс «богатый клиент уходит».  
→ Банку стоит запустить премиальный тариф и персональные retention‑акции для high‑balance сегмента.
